# DCA - Dynamic Channel Allocation with Reinforcement Learning

In [6]:
%autoreload 2
%cd mindca2

/home/torstein/code/mindca2/mindca2


Denne notebooken demonstrere en veldig forenklet versjon av problemstillingen om allokering av radiokanaler i mobilnettverk ved bruk av Reinforcement Learning.

Et mobilnettverk består av et område som er dekket av en gruppe med basestasjoner. Hver basestasjon dekker et begrenset område kalt en celle, hvor den gir dekning. Mobilnettverket har tilgang til et begrenset sett med radiokanaler. Hvis to basestasjoner som er i nærheten av hverandre bruker samme kanal, oppstår det interferens for mobilbrukerene. Man må derfor unngå å bruke samme kanal i celler som er nær hverandre:

![](fig/grid-reusedist.png "Reuse distance of 2")

Denne problemoppgaven tar for seg geografisk område som vi approksimerer med et grid bestående av heksagonale celler.
Vi antar at radiokanalar kan gjenbruker når avstanden mellom cellene som bruker samme kanal er 3 eller større. 

Hvis vi bruker en kanal i den mørkegrå cellen, må vi dermed unngå å bruke den i nærliggende naboer (grå celler).

![](fig/grid-rhombusaxial-neighs-noidx.png)

In [13]:
from grid import Grid
??Grid

Init signature: Grid(rows: int, cols: int, n_channels: int, *args, state=None, **kwargs)
Source:        
class Grid:
    """
    Rhombus grid with an axial coordinate system.
    """

    def __init__(
        self, rows: int, cols: int, n_channels: int, *args, state=None, **kwargs
    ):
        self.rows = rows
        self.cols = cols
        self.n_channels = n_channels

        if state is None:
            # For each cell, for each channel, marks whether the channel is in use or not
            self.state = np.zeros((self.rows, self.cols, self.n_channels), dtype=bool)
        else:
            self.state = state

    @classmethod
    def from_arr[T: Grid](cls: type[T], gridarr: GridArr) -> T:
        """Construct from an existing numpy array"""
        return cls(*gridarr.shape, state=gridarr)

    def validate_reuse_constr(self) -> None:
        """
        Verify that the channel reuse constraint of 3 is not violated.
        The reuse constraint is violated if a channel that i

Vi simulerer systemet med en diskret event-simulator. Hver tidssteg kommer en event: enten kommer det en forespørsel om en samtale, eller så blir en samtale avsluttet. Oppgaven til agenten er å allokere kanaler. Når det er en forespørsel om å opprette en samtale må agenten velge en radiokanal, eller blokkere forespørselen vis det ikke er noen radiokanaler som kan bruker uten å skape interferens. Når en samtale avlsluttes, kan vi velge å flytte en pågående samtale til kanalen som nettop har blitt ledig. 

Formålet til agenten er å blokkere færrest mulige samtaler. Dette er ekvivalent til å maksimere tidsintegralet av anvendelsen av gridden.

![](fig/discrete-event-sim.png)

In [10]:
from eventgen import EventGen
??EventGen

Init signature: EventGen(rows, cols, call_rate, call_duration, *args, **kwargs)
Source:        
class EventGen:
    """
    Event generator
    """

    def __init__(self, rows, cols, call_rate, call_duration, *args, **kwargs):
        self.rows = rows
        self.cols = cols
        # Avg. time between arriving calls
        self.call_intertimes = 1 / call_rate
        self.call_duration = call_duration

    def arrival_event(self, t: float, cell: Cell) -> ArrivalEvent:
        """
        Generate a call arrival event indicating an arriving call at the given cell at an
        exponentially distributed time dt from t.

        """
        dt = np.random.exponential(self.call_intertimes)
        return ArrivalEvent(t + dt, cell)

    def termination_event(self, t: float, cell: Cell, ch: int) -> TerminationEvent:
        """
        Generate END event for a call

        Return tuple with
        - Event time
        - Type of call event (an END event)
        - Cell number to end a c

Den tradisjonelle måten for å løse dette problemet på, kalt Fixed Channel Allocation (FCA), partisjonerer alle basestasjoner inn 7, slik at en gitt basestasjon ikke har naboer innenfor gjenbruksdistansen som tilhører samme gruppe. Hver gruppe er tildelt 1/7 av alle radiokanaler. Denne partisjoneringen forsikrer oss om at hver basestasjon kan bruke alle kanaler den er tildelt, uten å skape interferens med naboer. 

Ulempen med denne metoden er at hver celle har like mange tildelte radiokanaler, og at tildelingen av radiokanaler til celler er statisk. Hvis det på et gitt tidspunkt er høyere trafikk i en celle kontra dens naboer, kunne gridden hatt flere pågående samtaler totalt, hvis cellen med høy trafikk hadde fått flere kanaler enn naboene sine.

![](fig/grid-labelled.png)

In [16]:
from strats import Strat
??Strat

Init signature:
Strat(
    pp: dict,
    eventgen: eventgen.EventGen,
    logger: logging.Logger,
    sanity_check: bool = True,
)
Source:        
class Strat(ABC):
    """
    A call environment simulator plus a strategy/agent for handling
    call events, that is, to assign channels on incoming calls and
    possible reassigning channels on terminating calls.
    """
    grid: Grid

    def __init__(
        self,
        pp: dict,
        eventgen: EventGen,
        logger: Logger,
        sanity_check: bool = True,
    ):
        """
        :param sanity_check: Whether or not to verify that the channel reuse constraint
            is not broken each iteration of the simulation
        """
        self.rows = pp["rows"]
        self.cols = pp["cols"]
        self.n_channels = pp["n_channels"]
        self.n_events = pp["n_events"]
        # A heap queue of sorted (time, call_event) pairs.
        # The queue is sorted such that the call events with the lowest event
        # time i

In [18]:
from strats import FAStrat
??FAStrat

Init signature: FAStrat(pp, *args, **kwargs)
Source:        
class FAStrat(Strat):
    """
    Fixed assignment (FA) channel allocation.
    The set of channels is partitioned, and the partitions are permanently
    assigned to cells so that all cells can use all the channels assigned
    to them simultaneously without interference.
    """

    def __init__(self, pp,  *args, **kwargs):
        self.grid = FixedGrid(**pp)
        super().__init__(pp, *args, **kwargs)

    def get_action(self, event: Event, *args) -> int | None:
        """
        On arrival events, the first unused nominal channel is selected.

        No reassignment is done on termination events; thus the channel
        of the call is selected for termination.
        """
        match event:
            case ArrivalEvent(t, cell):
                # When a call arrives in a cell,
                # if any pre-assigned channel (i.e. a nominal channel) is unused;
                # it is assigned, else the call is bloc

Vi skal i tillegg bygge en Reinforcement Learning agent, en dynamisk agent, som i hver celle har mulighet til å bruke hver eneste radiokanal fra hele spekteret. 

Generelt sett så fungerer en RL agent i dette systemet på følgende måte. Hvert tidssteg håndteres et event. Agenten har en verdifunksjon eller en policy som sier noe om hvor bra hver handling er i en gitt tilstand. Ved hjelp av denne velger agenten en handling - en radiokanal for å allokere til en inkommende samtale eller en kanal for om omallokere til en forlatende samtale. Når handlingen er utført får agenten en reward, som er en numerisk verdi som scorer hvor bra handlingen er. Denne rewarden brukes til å forbedre verdifunksjonen. Agenten er med andre ord et maskinlæringssystem som går igjennom kontinuerlig læring og forbedres hele tiden.

![](fig/rl-dca-cycle.png)

Den spesifikke RL agenten som vi skal lage er en form for Q-Learning som kalles SARSA (State - Action - Reward - next State - next Action).

Den er hentet fra Lilith and Dogançay (2004) (Reduced-state sarsa with channel reassignment for dy-
namic channel allocation in cellular mobile networks.)

SARSA er en verdifunksjon $Q$ som estimerer hvor mye reward en agent kommer til å tjene fremover i tiden ved å utføre handling $a_t$ i tilstand $s_t$. Systemtilstanden består av gridden plus nåværende event. $ Q(s_t, a_t) $ er altså et estimat på summen av rewards fra tid $t$ fremover.

$ Q(s_t, a_t) = Q(s_t, a_t) + α*(r_t + γ*Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)) $

hvor $α$ er læringsraten og $γ$ er en factor, typisk 0.9-0.999, for å begrense Q-verdiene i et system hvor tidhorisonten kan være uendelig.

Denne formelen oppdaterer nåværende tilstands-handlings-estimat mot et oppdatert - og forbedret estimat som inkluderer rewarden fra sist utførte handling. 

Forbedret estimat av Q-verdien til nåværende tilstand-handling:

$ r_t + γ*Q(s_{t+1}, a_{t+1}) $



In [21]:
from strats import RLStrat
??RLStrat

Init signature: RLStrat(pp: dict, *args, **kwargs)
Docstring:     
A call environment simulator plus a strategy/agent for handling
call events, that is, to assign channels on incoming calls and
possible reassigning channels on terminating calls.
Source:        
class RLStrat(Strat):
    def __init__(self, pp: dict, *args, **kwargs):
        self.grid = Grid(**pp)
        super().__init__(pp, *args, **kwargs)
        self.exploration_policy = Greedy()
        self.losses = [0]
        self.gamma = pp['gamma']  # discount factor

    def get_initial_action(
        self,
        event: Event,
    ) -> int | None:
        """
        Return a channel to be assigned in response to 'cevent'.
        """
        # Choose A from S
        next_ch, next_max_ch = self.optimal_ch(event)
        return next_ch

    def get_action(
        self,
        next_event: Event,
        gridarr: GridArr,
        cell: Cell,
        ch: int | None,
        event_type: Type[Event]
    ) -> int | None:
    

In [25]:
from strats import QTable
??QTable

Init signature: QTable(pp, *args, **kwargs)
Docstring:     
A call environment simulator plus a strategy/agent for handling
call events, that is, to assign channels on incoming calls and
possible reassigning channels on terminating calls.
Source:        
class QTable(RLStrat):
    def __init__(self, pp, *args, **kwargs):
        super().__init__(pp, *args, **kwargs)
        # Learning rate for RL algorithm
        self.alpha = pp["alpha"]
        self.alpha_decay = pp["alpha_decay"]
        self.exploration_policy = Boltzmann(
            epsilon=pp["epsilon"], epsilon_decay=pp["epsilon_decay"]
        )
        self.qvals = np.zeros(self.grid.state.shape)

    @abstractmethod
    def feature_rep(self, cell: Cell, *args, **kwargs):
        """Feature representation of state"""
        raise NotImplementedError()

    def get_qvals(self, cell, chs, *args, **kwargs):
        """Get Q-Values for the given cell and all the given channels"""
        frep = self.feature_rep(cell)
        ret

Tilstandsrommet til gridden grenser opp i mot $(7*7*70)^2 = 11764900$ verdier og er dermed enormt. For å lære ett estimat for hver mulige tilstand
kan det dermed ta opp i mot 10 millioner steg. For å redusere størrelsen på tilstandsrommet, og dermed antall Q-verdier som må læres, bruker vi en forenklet representasjon av tilstanden. $x_t ~ s_t$ hvor $x_t$ er cellen / basestasjonen. Denne forenklingen gjør at hver celle har 70 Q-verdier - en for hver kanal - og tilstandsrommet er redusert fra 10M til rundt 3000.



In [27]:
from strats import RS_SARSA
??RS_SARSA

Init signature: RS_SARSA(*args, **kwargs)
Source:        
class RS_SARSA(QTable):
    """
    Reduced-state SARSA.
    State consists of cell coordinates only.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def feature_rep(self, cell: Cell, *args, **kwargs):
        return cell
File:           ~/code/mindca2/mindca2/strats.py
Type:           ABCMeta
Subclasses:     

hvordan RL kan være nyttig for å fobedre andre ML systemer - eksempelvis recommender systems eller LLMer (RLHF) hvor RL kan brukes til å forbedre systemet ved at brukeren gir en enkel feedback (en reward - f.eks en score fra 0 til 10). Det er mye raskere og enklere for brukerne av systemet til å gi en score på hvor god en LLM respons er kontra det å skrive ut en full LLM respons for å bruke til supervised learning. 